# Veri Analizleri

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
from plotly import express
from plotly import graph_objects
from plotly import io

io.templates.default = "plotly_dark"

In [4]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.stattools import adfuller, acf, breakvar_heteroskedasticity_test

In [5]:
from statsmodels.graphics import tsaplots
import matplotlib.pyplot as plt

In [6]:
import plotly.graph_objects as go
import statsmodels.api as sm 

In [7]:
country_name = ["France", "Greece",	"Italy", "Portugal", "Spain", "Türkiye"]

In [8]:
df_2 = pd.read_excel("OECD VERİLERİ ELİF BAĞCI TEZ V0_python.xlsx", sheet_name=5, header=1 )
df_2.head()

,Unnamed: 0,France,Greece,Italy,Portugal,Spain,Türkiye,t,interruption,interrupted t
0,2000,2119.934,874.121,1474.943,1146.403,1087.549,266.447,1,0,0
1,2001,2268.942,1075.096,1605.676,1200.502,1161.747,317.992,2,0,0
2,2002,2495.624,1179.607,1718.586,1324.828,1281.214,363.852,3,0,0
3,2003,2407.666,1226.410,1724.650,1357.830,1432.777,362.960,4,0,0
4,2004,2498.276,1231.573,1868.774,1472.358,1514.798,400.523,5,0,0


In [9]:
def create_corr_plot(series, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,13])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title)
    fig.show()

In [10]:
for country in country_name:
    print("###########################")
    print(f"{country}'nin Zaman Serisi Analizi")
    print("###########################")

    trendlin = express.scatter(
        df_2,
        x="t",
        y=f"{country}",
        title=f"{country}'s Government and compulsory health insurance schemes, per capita expenditure, US$ purchasing power parities (current prices, current PPPs))",
        trendline = "ols"
    )

    trendlin.show()
    linear_model = ols(f"{country} ~ t", data=df_2).fit()
    print(linear_model.summary())
    print(" ", end="\n")
    print("###########################")
    print("Verinin Durağanlık Kontrolü", end="\n\n")
    print("H0: Seri durağan değildir.")
    print("H1: Seri durağandır.", end="\n\n")
    adfullerx = adfuller(df_2[[f"{country}"]])
    print(f'Test Stat = {round(adfullerx[0], 4)}, p-value = {round(adfullerx[1], 4)}')

    if adfullerx[1]<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Yani seri durağandır.")
    elif adfullerx[1]>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani seri durağan değildir.", end="\n\n")
    print("###########################")
    print("Hataların Normal Dağılması", end="\n\n")
    residual_values = linear_model.resid

    print("H0: Normal dağılım varsayımı sağlanmaktadır.")
    print("H1: Normal dağılım varsayımı sağlanmamaktadır.", end="\n\n")

    test_stat, pvalue = stats.shapiro(residual_values)
    print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue, 4)}')
    if pvalue<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar normal dağılmamıştır.")
    elif pvalue>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar normal dağılmıştır.", end="\n\n")
    hist = express.histogram(residual_values, title="Hataların Dağılımı")
    hist.show()

    print("###########################")
    print("Hataların Rassallığı", end="\n\n")

    print("     Hataların Durağanlığı", end="\n\n")

    adfullery = adfuller(residual_values)
    print(f'Test Stat = {round(adfullery[0], 4)}, p-value = {round(adfullery[1], 4)}')

    if adfullery[1]<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.")
    elif adfullery[1]>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.", end="\n\n")

    resid_scatter = express.scatter(
        df_2,
        x="t",
        y=residual_values,
        title="Hataların Rassallığı"
    )

    resid_scatter.show()


    print("     Hataların Otokorelasyonu", end="\n\n")

    print("Ljung-Box q istatistiğinin Hipotezleri")
    print("H0: Veriler bağımsız (random) olarak dağılmıştır.")
    print("H1: Veriler bağımsız (random) olarak dağılmamıştır.", end="\n\n")

    acf_df = pd.DataFrame(pd.DataFrame(acf(residual_values, qstat=True)).T)
    acf_df.columns = ["ACF", "Ljung-Box q statistic", "p-value"]
    display(acf_df[1:6])

    create_corr_plot(residual_values)


###########################
France'nin Zaman Serisi Analizi
###########################


                            OLS Regression Results                            
Dep. Variable:                 France   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                     678.6
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           2.49e-16
Time:                        19:06:40   Log-Likelihood:                -131.03
No. Observations:                  21   AIC:                             266.1
Df Residuals:                      19   BIC:                             268.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1888.9961     58.997     32.018      0.0

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -1.9317, p-value = 0.3173
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.568723,11.271986,0.003567
2,0.368969,12.674596,0.005396
3,0.228629,12.741722,0.012609
4,0.048607,13.793256,0.016977
5,-0.186637,15.854641,0.014556


...

###########################
Türkiye'nin Zaman Serisi Analizi
###########################


                            OLS Regression Results                            
Dep. Variable:                Türkiye   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     1863.
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           1.98e-20
Time:                        19:06:40   Log-Likelihood:                -95.685
No. Observations:                  21   AIC:                             195.4
Df Residuals:                      19   BIC:                             197.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    225.7862     10.963     20.595      0.0

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -3.0971, p-value = 0.0268
p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.


     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.423679,4.458529,0.107608
2,0.069704,8.647678,0.034361
3,-0.395117,17.440032,0.001587
4,-0.556293,21.294839,0.000712
5,-0.357345,22.431220,0.001011


# Interrupted Time Series for Türkiye

    Yukarıda referans ülkelerin ve Türkiye'nin zaman serisi regresyon analizi sonuçlarına bakılırken, bu kısımdan itibaren Türkiye için interrupted time series analysis yapılmıştır.

In [13]:
X = df_2[["t", "interruption", "interrupted t"]]
y = df_2["Türkiye"]

X = sm.add_constant(X) 

analys= sm.OLS(y, X).fit()

trendlin = express.scatter(
        df_2,
        x="t",
        y= "Türkiye",
        title="Türkiye's Government and compulsory health insurance schemes, per capita expenditure, US$ purchasing power parities (current prices, current PPPs)",
    )

trendlin.add_trace(
go.Scatter(
    x = [1,5],
    y = [analys.predict()[0], analys.predict()[4]],
)
 )   

trendlin.add_trace(
go.Scatter(
    x = [5,6],
    y = [analys.predict()[4], analys.predict()[5]]
)

 )

trendlin.add_trace(
go.Scatter(
    x = [6,21],
    y = [analys.predict()[5], analys.predict()[20]]
)

 )  

trendlin.show() 



print(analys.summary())

print("###########################")
print("Hataların Normal Dağılması", end="\n\n")
residual_values = analys.resid
print("H0: Normal dağılım varsayımı sağlanmaktadır.")
print("H1: Normal dağılım varsayımı sağlanmamaktadır.", end="\n\n")
test_stat, pvalue = stats.shapiro(residual_values)
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue, 4)}')
if pvalue<0.05:
    print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar normal dağılmamıştır.")
elif pvalue>0.05:
     print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar normal dağılmıştır.", end="\n\n")
hist = express.histogram(residual_values, title="Hataların Dağılımı")
hist.show()
print("###########################")
print("Hataların Rassallığı", end="\n\n")
print("     Hataların Durağanlığı", end="\n\n")
adfullery = adfuller(residual_values)
print(f'Test Stat = {round(adfullery[0], 4)}, p-value = {round(adfullery[1], 4)}')
if adfullery[1]<0.05:
    print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.")
elif adfullery[1]>0.05:
    print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.", end="\n\n")
resid_scatter = express.scatter(
    df_2,
    x="t",
    y=residual_values,
    title="Hataların Rassallığı"
)
resid_scatter.show()
print("     Hataların Otokorelasyonu", end="\n\n")
print("Ljung-Box q istatistiğinin Hipotezleri")
print("H0: Veriler bağımsız (random) olarak dağılmıştır.")
print("H1: Veriler bağımsız (random) olarak dağılmamıştır.", end="\n\n")
acf_df = pd.DataFrame(pd.DataFrame(acf(residual_values, qstat=True)).T)
acf_df.columns = ["ACF", "Ljung-Box q statistic", "p-value"]
display(acf_df[1:6])
create_corr_plot(residual_values)

                            OLS Regression Results                            
Dep. Variable:                Türkiye   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     591.4
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           2.19e-17
Time:                        19:07:46   Log-Likelihood:                -95.037
No. Observations:                  21   AIC:                             198.1
Df Residuals:                      17   BIC:                             202.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           248.4188     26.047      9.537

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -1.3429, p-value = 0.6093
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.378838,3.559091,0.168715
2,0.060523,6.807829,0.078282
3,-0.347953,13.114364,0.010730
4,-0.471136,14.449513,0.012993
5,-0.210306,15.043867,0.019918
